In [37]:
#audible_data = spark.sql("select * from result")
dt_clean = spark.read.csv('gs://dataproc-staging-asia-southeast2-537169427832-9rlydjvh/google-cloud-dataproc-metainfo/f0bb62b5-0e85-4a7c-9fa0-c407c0582d85/cluster-awoy-m/ws2_data.csv', header = True, inferSchema = True, )

In [38]:
from pyspark.sql import functions as f

dt_clean = dt.withColumn("timestamp",
                        f.to_timestamp(dt.timestamp, 'yyyy-MM-dd HH:mm:ss')
                        )
dt_clean.show()

+-------------------+--------+-------+--------------------+-----+
|          timestamp| user_id|book_id|             country|price|
+-------------------+--------+-------+--------------------+-----+
|2021-05-01 08:03:44|824dc295|      2|              Israel|24.95|
|2021-05-02 02:48:34|f80fe106|      6|            Thailand|31.49|
|2021-05-01 16:54:57|01cb9fe0|      9|               Italy|29.65|
|2021-05-01 21:45:28|59508f47|     13|        South Africa| 22.4|
|2021-05-01 14:50:52|3553ebbb|     14|              Canada| 28.0|
|2021-05-01 08:41:48|ea28260a|     16|United States of ...|31.18|
|2021-05-01 05:51:43|ff44767e|     18|United States of ...|59.99|
|2021-05-01 00:01:09|b07799e7|     20|              Canada| 28.0|
|2021-05-01 00:21:24|9d2e93c8|     20|             Hungary| 28.0|
|2021-05-01 01:34:23|612db483|     20|              Sweden| 28.0|
|2021-05-01 01:42:10|afad01ae|     20|              France| 28.0|
|2021-05-01 03:38:00|3b68cda1|     20|              Canada| 28.0|
|2021-05-0

In [39]:
# สำคัญ: เปลี่ยน aaa เป็นชื่อประเทศที่คุณคิดว่าผิด และ bbb เป็นชื่อประเทศที่ถูกต้อง
from pyspark.sql.functions import when

dt_clean_country = dt_clean.withColumn("CountryUpdate", when(dt_clean['Country'] == 'Japane', 'Japan').otherwise(dt_clean['Country']))
dt_clean = dt_clean_country.drop("Country").withColumnRenamed('CountryUpdate', 'Country')

In [40]:
dt_clean.show()

+-------------------+--------+-------+-----+--------------------+
|          timestamp| user_id|book_id|price|             Country|
+-------------------+--------+-------+-----+--------------------+
|2021-05-01 08:03:44|824dc295|      2|24.95|              Israel|
|2021-05-02 02:48:34|f80fe106|      6|31.49|            Thailand|
|2021-05-01 16:54:57|01cb9fe0|      9|29.65|               Italy|
|2021-05-01 21:45:28|59508f47|     13| 22.4|        South Africa|
|2021-05-01 14:50:52|3553ebbb|     14| 28.0|              Canada|
|2021-05-01 08:41:48|ea28260a|     16|31.18|United States of ...|
|2021-05-01 05:51:43|ff44767e|     18|59.99|United States of ...|
|2021-05-01 00:01:09|b07799e7|     20| 28.0|              Canada|
|2021-05-01 00:21:24|9d2e93c8|     20| 28.0|             Hungary|
|2021-05-01 01:34:23|612db483|     20| 28.0|              Sweden|
|2021-05-01 01:42:10|afad01ae|     20| 28.0|              France|
|2021-05-01 03:38:00|3b68cda1|     20| 28.0|              Canada|
|2021-05-0

In [41]:
dt_clean.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- Country: string (nullable = true)



In [42]:
dt_clean.where(dt_clean['user_id'].rlike("^[a-z0-9]{8}$")).count()

499996

In [43]:
dt_correct_userid = dt_clean.filter(dt_clean["user_id"].rlike("^[a-z0-9]{8}$"))
dt_incorrect_userid = dt_clean.subtract(dt_correct_userid)

dt_incorrect_userid.show(10)

+-------------------+----------+-------+-----+--------------------+
|          timestamp|   user_id|book_id|price|             Country|
+-------------------+----------+-------+-----+--------------------+
|2021-05-01 14:41:20|      null|    516|27.37|              Brazil|
|2021-06-30 02:27:00|      null|   1390|27.99|               Japan|
|2021-06-19 16:00:22|ca86d17200|   2269|35.71|United States of ...|
|2021-07-15 18:53:09|ca86d17200|    322|30.79|United States of ...|
+-------------------+----------+-------+-----+--------------------+



In [44]:
dt_clean_userid = dt_clean.withColumn("user_id_update", when(dt_clean['user_id'] == 'ca86d17200', 'ca86d172').otherwise(dt_clean['user_id']))

In [45]:
dt_correct_userid = dt_clean_userid.filter(dt_clean_userid["user_id"].rlike("^[a-z0-9]{8}$"))
dt_incorrect_userid = dt_clean_userid.subtract(dt_correct_userid)
dt_clean = dt_clean_userid.drop("user_id").withColumnRenamed('user_id_update', 'user_id')

In [46]:
dt_clean_userid = dt_clean.withColumn("user_id_update", when(dt_clean['user_id'].isNull(), '00000000').otherwise(dt_clean['user_id']))
dt_clean = dt_clean_userid.drop("user_id").withColumnRenamed('user_id_update', 'user_id')

In [47]:
dt_clean.where( dt_clean.user_id.isNull() ).show()

+---------+-------+-----+-------+-------+
|timestamp|book_id|price|Country|user_id|
+---------+-------+-----+-------+-------+
+---------+-------+-----+-------+-------+



In [50]:
dt_clean.write.mode("overwrite").csv('gs://processed_awoy/Cleaned_data.csv', header = True)

In [1]:
print("processed finnish awoy12334")

processed finnish awoy
